# ECON 0150 | Replication Notebook

**Title:** Minimum Wage and Fast Food Employment

**Original Authors:** Hargraves; Bisram

**Original Date:** Fall 2024

---

This notebook replicates the analysis from a student final project in ECON 0150: Economic Data Analysis.

## About This Replication

**Research Question:** How does Minimum Wage affect the Fast Food Industry?

**Data Source:** State-level minimum wage and fast food employment data (51 states/DC)

**Methods:** OLS regression: Employment ~ Min_Wage

**Main Finding:** Cross-sectional analysis of minimum wage and fast food employment across states.

**Course Concepts Used:**
- Simple linear regression
- Cross-sectional analysis
- Scatter plots with regression lines
- Labor economics

---
## Step 0 | Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

In [ ]:
# Load data from course website
base_url = 'https://tayweid.github.io/econ-0150/projects/replication/replications/0061/data/'

data = pd.read_excel(base_url + 'econfinaldata.xlsx')

print(f"Number of states: {len(data)}")
data.head(10)

---
## Step 1 | Data Preparation

In [ ]:
# Check columns
print("Columns:", data.columns.tolist())

# Rename column for easier use
data = data.rename(columns={'Min Wage': 'Min_Wage'})

print(f"\nStates with $0 minimum wage (using federal): {len(data[data['Min_Wage'] == 0])}")

In [ ]:
# States with $0 use federal minimum wage ($7.25)
# Create effective minimum wage variable
data['Effective_Min_Wage'] = data['Min_Wage'].apply(lambda x: 7.25 if x == 0 else x)

# Employment in thousands
data['Employment_Thousands'] = data['Employment'] / 1000

print(f"\nData prepared: {len(data)} observations")
data.head()

---
## Step 2 | Data Exploration

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(data[['Effective_Min_Wage', 'Employment']].describe())

In [ ]:
# Correlation
correlation = data['Effective_Min_Wage'].corr(data['Employment'])
print(f"\nCorrelation between minimum wage and employment: {correlation:.4f}")

In [ ]:
# States with highest/lowest minimum wages
print("\nHighest minimum wage states:")
print(data.nlargest(5, 'Effective_Min_Wage')[['State', 'Effective_Min_Wage', 'Employment']])

print("\nLowest minimum wage states:")
print(data.nsmallest(5, 'Effective_Min_Wage')[['State', 'Effective_Min_Wage', 'Employment']])

---
## Step 3 | Visualization

In [ ]:
# Distribution of minimum wages
plt.figure(figsize=(10, 6))
sns.histplot(data['Effective_Min_Wage'], kde=True, bins=15)
plt.title('Distribution of State Minimum Wages')
plt.xlabel('Effective Minimum Wage ($)')
plt.ylabel('Frequency')
plt.axvline(7.25, color='red', linestyle='--', label='Federal Minimum ($7.25)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(x='Effective_Min_Wage', y='Employment_Thousands', data=data,
            scatter_kws={'s': 80, 'alpha': 0.7})
plt.title('Minimum Wage vs. Fast Food Employment by State')
plt.xlabel('Effective Minimum Wage ($)')
plt.ylabel('Fast Food Employment (Thousands)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Labeled scatter plot for large states
plt.figure(figsize=(12, 8))
plt.scatter(data['Effective_Min_Wage'], data['Employment_Thousands'], s=80, alpha=0.7)

# Label largest states
for idx, row in data.nlargest(8, 'Employment').iterrows():
    plt.annotate(row['State'], (row['Effective_Min_Wage'], row['Employment_Thousands']),
                fontsize=9, alpha=0.8)

plt.xlabel('Effective Minimum Wage ($)')
plt.ylabel('Fast Food Employment (Thousands)')
plt.title('Minimum Wage vs. Fast Food Employment (Large States Labeled)')
plt.grid(True, alpha=0.3)
plt.show()

---
## Step 4 | Statistical Analysis

In [ ]:
# OLS Regression
model = smf.ols('Employment ~ Effective_Min_Wage', data=data).fit()
print("OLS Regression: Employment ~ Effective_Min_Wage")
print(model.summary())

In [ ]:
# Key results
print("\n" + "="*50)
print("KEY RESULTS")
print("="*50)
print(f"\nNull Hypothesis: Minimum wage does not affect employment (beta = 0)")
print(f"\nModel Results:")
print(f"  Intercept: {model.params['Intercept']:.0f} employees")
print(f"  Min Wage coefficient: {model.params['Effective_Min_Wage']:.2f}")
print(f"  P-value: {model.pvalues['Effective_Min_Wage']:.4f}")
print(f"  R-squared: {model.rsquared:.3f}")
print(f"\nInterpretation:")
print(f"  Each $1 increase in minimum wage is associated with")
print(f"  {model.params['Effective_Min_Wage']:.0f} change in fast food employment")

In [ ]:
# Note on interpretation
print("\n" + "="*50)
print("IMPORTANT CAVEAT")
print("="*50)
print("\nThis cross-sectional analysis is HIGHLY CONFOUNDED!")
print("\nLarge states (CA, TX, FL, NY) have:")
print("  - More fast food employment (due to larger population)")
print("  - Often higher minimum wages (especially CA, NY)")
print("\nPopulation is a major omitted variable.")
print("Without controlling for state size, results are misleading.")

In [ ]:
# Better analysis: Employment per capita (if population data available)
# Or use log transformation
data['Log_Employment'] = np.log(data['Employment'] + 1)

model_log = smf.ols('Log_Employment ~ Effective_Min_Wage', data=data).fit()
print("\nWith Log-Transformed Employment:")
print(model_log.summary().tables[1])

---
## Step 5 | Results Interpretation

### Key Findings

1. **Positive Correlation:** Higher minimum wage states have more fast food employment

2. **But This Is Confounded:** Population drives both variables

### The Omitted Variable Problem

States like California and New York have:
- High minimum wages
- Large populations
- Therefore, more total employment in every industry

This creates a **spurious positive correlation** between minimum wage and employment.

### What the Economics Literature Says

The minimum wage debate is one of the most contested in economics:

- **Traditional view:** Higher minimum wage reduces employment (standard supply/demand)
- **Card-Krueger (1994):** Famous study found no negative employment effects in fast food
- **Recent research:** Mixed findings, with effects depending on local labor market conditions

### What Would Be Needed for Causal Inference?

- Panel data (multiple years)
- Control for population/state size
- Difference-in-differences design
- Compare bordering counties with different minimum wages

---
## Replication Exercises

### Exercise 1: Per Capita Analysis
Find state population data and calculate fast food employment per 1000 residents.

### Exercise 2: Regional Analysis
Group states by region. Do minimum wage effects differ by region?

### Exercise 3: Above/Below Federal
Compare states at federal minimum vs. above. Is there a difference in employment?

### Challenge Exercise
Read the Card-Krueger study on New Jersey's minimum wage increase. What methodology did they use?

In [ ]:
# Your code for exercises

# Example: Compare federal minimum vs. above
# data['Above_Federal'] = (data['Effective_Min_Wage'] > 7.25).astype(int)
# print(data.groupby('Above_Federal')['Employment'].mean())